## Author: Etienne Bonnassieux
## Comments, questions? Contact etienne.bonnassieux@unibo.it

# **I. Prerequisites**

In this jupyter notebook, we will learn how to perform a round of self-calibration using killMS & DDFacet on a LOFAR dataset. All the required software will be pre-installed on the singularity you'll be using for the school, and the data you will work on will be provided alongside it. If you want to do this later on your own machine and in your own time, you must first have the following installed (non-exhaustive list, I probably forgot at least one thing):

- the KERN suite http://kernsuite.info/installation/
- ddfacet - sudo apt-get install ddfacet; pip install ephem
- dysco (if using a LOFAR dataset) - sudo apt-get install dysco
- killMS, which you can acquire through saopicc: https://github.com/saopicc/killMS

taking care to follow the instructions in individual gits where appropriate. Note that all the installs recommended here only work after installing KERN. If you don't want to use KERN, that is fine! Just be prepared for a good week of tears and sorrow. Note that KERN also comes with a lot of other useful radio-astro tools; I personally recommend tigger as an image viewer (takes a while to get used to but has some very useful functions) and pybdsf if you wish to get deeper than this tutorial in interferometric imaging.

Note that you can also get the Wirtinger suite directly from the public release repositories:

* https://github.com/saopicc/DDFacet
* https://github.com/saopicc/killMS

and follow the installation instructions there. There are usually a host of problems with that sort of installation, as with all radio-astronomy software. KERN allows you to avoid them, to an extent. Note that you can find the best "documentation" for DDFacet (and similarly for killMS, to a much lesser degree) in their respective parset.cfg file. They can be found here:

https://github.com/saopicc/DDFacet/blob/master/DDFacet/Parset/DefaultParset.cfg
https://github.com/saopicc/killMS/blob/master/Parset/DefaultParset.cfg


Finally, should you want a slightly more convenient install than the above, Martin Hardcastle's DDF-pipeline installs them for you:

* https://github.com/mhardcastle/ddf-pipeline

For the purposes of the LOFAR school, the necessary software was installed as part of a local singularity by our talented and handsome colleagues in IT; the information above is therefore included for those users who wish to use this software to reduce data on machines they are responsible for.

# **II. Launching the singularity**

The first thing we'll need to do is to set up the singularity properly so as to be able to carry out this tutorial session. The way to do this is specific to the Bologna cluster for now, as far as I'm aware, but efforts to standardise this approach across all italian LOFAR computing infrastructures are ongoing. Enter the following commands in bash. If they are in separate cells, they can be entered in clusters but not all at once (i.e. enter all the commands for one cell first, then all commands for the next, and so on).

In [ ]:
lofar-setup  190516.17pub
. /irasoft/common/irainit/irainit_setup-alias.sh
singularity-setup
lofar-singularity

In [ ]:
source /opt/lofar-cfg.sh
source /opt/lofar-init.sh

# **II. Introduction**


The aim of this tutorial is to calibrate & image a small LOFAR dataset using the so-called "Wirtinger Pack": DDFacet and killMS. Once you have finished this tutorial, you should have some basic understanding of how to run these software packages to reduce & image your own datasets. Here is the order of operations we will follow:

- Inspecting the dataset
- Making a first image using DDFacet
- Making a model from this image
- Perform a direction-independent calibration run on our data with our model
- Imaging the newly-calibrated data
- Do the same with more than 1 direction

These are two separate self-calibration loops: one is direction-independent, and one is direction-dependent. We'll begin by creating a DATA folder, putting one of our pointing's dataset there, and loading the singlarity. Type the following commands in your command terminal:

In [ ]:
mkdir DATA
cp /iranet/home2/lofar/botteon/LOFAR_SCHOOL_DR1/GSM_CAL_L232875_ABN_174.tar.gz DATA
cd DATA
tar -xvf GSM_CAL_L232875_ABN_174.tar.gz
rm *MS/*lock
cd ..

We have thus created a DATA directory where we've moved our downloaded dataset, untarred it, removed the lock, and then we returned to our working directory. It can be inspected with tools like msoverview, casaviewer, or the pyrap packages; these are generally dependent on the casacore software. Let's inspect the dataset with pyrap to see what columns we have in there.

In [ ]:
from pyrap.tables import table
t=table("prefactor/results/L232875_SB070_uv.dppp.pre-cal_124B2FCD4t_134MHz.pre-cal.ms/")
t.colnames()

As we can see, we have a number of columns in this dataset. In particular, we have "DATA" and "CORRECTED_DATA". This tells us that the data was calibrated already; in this case, it was calibrated with prefactor in amplitude and phase. Let's image this calibrated data.

# **IV. DI Self-cal - First step: imaging prefactored data with DDFacet**


Here, we will explore using DDFacet to image some interferometric data contained in a Measurement Set, the standard dataset for LOFAR. Let's start by taking a little look at the options available. Run the following command:

In [ ]:
DDF.py -h

As you can see, there's quite a few!...in fact, the vast majority of them can be safely ignored. For starters, let's make a small image with our dataset. Since we're running this on LOFAR surveys data, for which the software was developed, we can safely run it with mostly default options. We will use three non-default options here: first, we specify the primary beam of our observation, as this will allow us to get images in apparenty and integrated flux (--Beam-Model LOFAR). We will also specify the number of CPUs to use explicitly, to avoid taxing the machine too much (--Parallel-NCPU 4). Finally, since there is no default for the dataset to be imaged, we specify it directly.

In [ ]:
 DDF.py --Data-MS prefactor/results/L232875_SB070_uv.dppp.pre-cal_124B2FCD4t_134MHz.pre-cal.ms/ --Parallel-NCPU 4 --Beam-Model LOFAR

If all went well, you should now have a restored image of our prefactored data. Let's take a look at a few of the images created by our run here: by default, these will all have the prefix "image" and be in the current directory. We can inspect the fits files with ds9:

In [ ]:
ds9 image*fits

This should open a lot of different images. The ones you are most likely to want to inspect are:

- image.int.restored.fits (if you applied the LOFAR beam)
- image.dirty.fits        (to see if anything was catastrophically wrong with the data)
- image.int.residual.fits (to see what your CLEAN components are)

in this example, because we specified a primary beam model, it was automatically applied to create the .int files (which show the _integrated_ flux in Jy/beam). The .app files files (which show the _apparent_ flux) are also created by default. Of course, if our observation was made with another instrument, we would not specify --Beam-Model LOFAR; instead, we would provide a .fits file which gives the value of the primary beam attenuation as a function of distance from pointing centre. This is beyond the scope of this tutorial, however, but know that it is supported by DDFacet

Note that our run also created a file called image.parset, which contains all the options used for the creation of that image. This is very convenient - it not only allows you to easily replicate a given run (changing only the salient parameters in the command line) but also serves as a "history" of what you actually did, and is therefore a good place to check for unexpected parameters if your images have unexpected problems. Shown below are the parameters which are most probably important to you, along with a short commentary giving tips for it.

In [ ]:
[Data]
MS = mydataset.MS                          # name of your ms. No default value.
ColName = CORRECTED_DATA                   # name of column to image
ChunkHours = 0.0                           # in case of memory error, set to 4, 2, 1...until it works

[Selection]
FlagAnts =                                 # regular expression or strings for ants to flag
UVRangeKm = [0, 2000]                      # self evident

[Output]
Mode = Clean                               # can also be Dirty or PSF for different test cases
Name = image                               # name of image you're making
RestoringBeam = None                       # I would advise manually setting this to 4 times cell size
Cubes =                                    # if you want freq. cubes. See -h
Images = DdPAMRIikz                        # see DDF.py -h | grep Images 

[Image]
NPix = 5000                                # number of pixels a side
Cell = 5.0                                 # pixel size in arcsec
PhaseCenterRADEC = align                   # where to put centre of image. If align, same as MS phase centre

[Weight]
ColName = WEIGHT_SPECTRUM                  # weight column to use
Mode = Briggs                              # self evident but maybe you want to use uniform or something
Robust = 0.0                               # see above

[RIME]
DecorrMode =                               # set to FT for wide fields of view. it's RIME magic.

[Parallel]
NCPU = 4                                   # number of CPUs used for parallelisation

[Cache]
Reset = True                               # good practice to set this to True when not changing image name.

[Beam]
Model = LOFAR                              # can be LOFAR or None. If you image eg GMRT/VLA data, provide fits beam file instead.
LOFARBeamMode = AE                         # can be A or AE

[Freq]
NBand = 1                                  # use this when you want to make cubes: get as many freq. slices as NBand

# after that, it's just CLEAN options.

[Deconv]
Mode = HMP 
MaxMajorIter = 20 
MaxMinorIter = 20000 
AllowNegative = True 
Gain = 0.1 
FluxThreshold = 0.0 
CycleFactor = 0.0 
RMSFactor = 0.0 
PeakFactor = 0.15 
PrevPeakFactor = 0.0 
NumRMSSamples = 10000 
ApproximatePSF = 0 
PSFBox = auto 

[Mask]
External = None 
Auto = False 
SigTh = 10 
FluxImageType = ModelConv 


[HMP]
Alpha = [-1.0, 1.0, 11] 
Scales = [0] 
Ratios = [''] 
NTheta = 6 
SolverMode = PI 
AllowResidIncrease = 0.1 
MajorStallThreshold = 0.8 
Taper = 0 
Support = 0 
PeakWeightImage = None 
Kappa = 0.0 
OuterSpaceTh = 2.0 
FractionRandomPeak = None 

 Let's use the parset to redo our run, changing only the RIME-DecorrMode option to FT since we are making a widefield image.

In [ ]:
DDF.py image.parset --RIME-DecorrMode FT

Now, it should have crashed, and for quite a good reason: we asked DDF to load a parset, and then overwrite it in the same run. This is a terrible idea, and so DDF doesn't allow it. Let's change the name of our output image too, then:

In [ ]:
DDF.py image.parset --RIME-DecorrMode FT --Output-Name image.ft

Note the structure of the command line options here, and compare it to the structure of the help message and the parset. For any given option, you give two dashes, then the main category (Beam, Output, Weight, Data, ...) comes first, then a dash, then the subcategory. The parset is organised in categories with a list of subcategories and associated values, as is the help. Capitalisation can be a bit haphazard, but that minor issue aside it is very easy to build your command line from the help and run with a parset so as to have either very long but complete command lines (best practice to avoid stupid mistakes, or at least maximise your chances of spotting them) or very short command lines (convenient but potentially a trap).

Regardless, we have our first image. We could change some more parameters, such as the number of pixels (--Image-NPix) or the size of a pixel in arcsec (--Image-Cell); since our image doesn't seem to have bright sources whose sidelobes pollute our field, we can leave it at that for now. Let us now move on to making a catalog from our image, which we will then use to calibrate our image.

# **V. DI Self-cal - Second step: making a model from our image**

This is a very quick step: there are many ways to go about it. Here, we will use a tool internal to the Wirtinger pack, called MakeModel. Know, however, that the same thing can be done using pybdsf and exporting the catalog as a "bbs"-format "gaul" - "bbs" being the name of a calibration solver used in prefactor, and gaul meaning GAUssian List. The advantage of using pybdsf is the power of the blob finder and the ability to group components into sources (can significantly speed up calibration time, especially for large extended emission). The drawback is that you don't use the CLEAN components directly; this can be a blessing and a curse.

# **VI. DI Self-cal - Third step: calibrating our data from our model**

bla bla

# **VII. DI Self-cal - Fourth step: Imaging our newly-selfcal'd data**

bla bla

# **VIII. DD Self-cal - First step: imaging DI-selfcal data with DDFacet**

bla bla

# **IX. DD Self-cal - Second step: making a model from our new image**

bla bla

# **X. DD Self-cal - Third step: calibrating our data from our model**

bla bla

# **VII. DD Self-cal - Fourth step: Imaging our newly-selfcal'd data**

bla bla